# License Plate Detection

In [1]:
# # installing missing YOLO dependencies
# %pip install lapx>=0.5.2
# # installing OCR library
# %pip install easyocr
# #installing CNN model
# %pip install tensorflow

# %pip install ultralytics

We are importing the following libraries:
* **ast** for parsing the bounding boxes
* **cv2** for video processing
* **easyocr** for OCR
* **glob** for finding files
* **numpy** for array operations
* **pandas** for dataframes
* **string** for string operations
* **ultralytics** for **YOLO** for object detection

In [2]:
import ast
import imutils
from glob import glob
import numpy as np
import pandas as pd
import string
from ultralytics import YOLO
from keras.models import load_model
import joblib
import cv2
import matplotlib.pyplot as plt

## License Plate Detection

**YOLOv8** is capable of detecting cars, buses and trucks very easily without additional trainings from the dataset.It is already trained from the COCO dataset.But license number plates seem to be a bit harder. The model often confuses street signs or just basic backgound noise as a car registration plate. 
<br/>
<br/>
To make things more efficient, we are combining both models - a regular COCO trained YOLOv8 and our number plate detector.If the COCO model spots a car, we will then execute the number plate detector to focus its search within the area marked out by the first model's bounding box. That way, we are only seaarching for number plates when there is a car in the picture.

This is a regular COCO trained YOLOv8 model for car detection.<br/>
`coco_model = YOLO('yolov8n.pt')`

This is our custom model trained on the License Plate Dataset.<br/>
`np_model = YOLO('../model/runs/detect/train/weights/best.pt')`

*best.pt* weight is produced by training our model with +21000 annoted images of license plates for 3 epochs.

In [3]:
coco_model = YOLO('yolov8n.pt')
np_model = YOLO('../model/runs/detect/train/weights/best.pt')
cnn_model = load_model('D:/Projects/anpr/notebooks/outputs/handwriting_recognition.keras')

TypeError: weight_decay is not a valid argument, kwargs should be empty  for `optimizer_experimental.Optimizer`.

The input video is read by glob. Glob is a function that returns all the pathnames matching a pattern.

In [ ]:
videos = glob('./inputs/sample.mp4')
print(videos)

['./inputs/sample.mp4']


### STEP 1 Implementing the Car Detection

Get the bounding boxes of all vehicles in our video recording with prediction confidence score and object tracking ID

This code currently gathers all the bounding boxes for vehicles in the video and stores them in the `vehicle_bounding_boxes` list. Along with the bounding box coordinates, this list also includes the tracking ID assigned to each identified vehicle. The tracking ID remains consistent from frame to frame, serving as a unique identifier. Additionally, the score indicates the model's confidence level that the particular bounding box indeed contains a vehicle, with values ranging from 0 to 1.

### STEP 2 Implementing the License Plate Detection

Use the bounding box for each vehicle and use the number plate detector model to try to find the corresponding plate within in the confinement of those boxes.

### STEP 3 Preprocess License Plates

### STEP 4 Read License Plates

In [ ]:
# write_csv is a function that writes the obtained results to a CSV file using the specified format.
# Here we are formatting the colunms as [frame_number, track_id, car_bbox, car_bbox_score, license_plate_bbox, license_plate_bbox_score, license_plate_number, license_text_score].
# car_bbox and license_plate_bbox has 4 array that stores the coordinate of the bounding box.

def write_csv(results, output_path):
    
    with open(output_path, 'w') as f:
        f.write('{},{},{},{},{},{},{},{}\n'.format(
            'frame_number', 'track_id', 'car_bbox', 'car_bbox_score',
            'license_plate_bbox', 'license_plate_bbox_score', 'license_plate_number',
            'license_text_score'))

        for frame_number in results.keys():
            for track_id in results[frame_number].keys():
                print(results[frame_number][track_id])
                if 'car' in results[frame_number][track_id].keys() and \
                   'license_plate' in results[frame_number][track_id].keys() and \
                   'number' in results[frame_number][track_id]['license_plate'].keys():
                    f.write('{},{},{},{},{},{},{},{}\n'.format(
                        frame_number,
                        track_id,
                        '[{} {} {} {}]'.format(
                            results[frame_number][track_id]['car']['bbox'][0],
                            results[frame_number][track_id]['car']['bbox'][1],
                            results[frame_number][track_id]['car']['bbox'][2],
                            results[frame_number][track_id]['car']['bbox'][3]
                        ),
                        results[frame_number][track_id]['car']['bbox_score'],
                        '[{} {} {} {}]'.format(
                            results[frame_number][track_id]['license_plate']['bbox'][0],
                            results[frame_number][track_id]['license_plate']['bbox'][1],
                            results[frame_number][track_id]['license_plate']['bbox'][2],
                            results[frame_number][track_id]['license_plate']['bbox'][3]
                        ),
                        results[frame_number][track_id]['license_plate']['bbox_score'],
                        results[frame_number][track_id]['license_plate']['number'],
                        results[frame_number][track_id]['license_plate']['text_score'])
                    )
        f.close()

### STEP 5 Clean-Up License Plate Format

This returns a list with bounding box metrics for every frame with a successful detection.

In [ ]:
def sort_contours(cnts, method="left-to-right"):
    reverse = False
    i = 0
    if method == "right-to-left" or method == "bottom-to-top":
        reverse = True
    if method == "top-to-bottom" or method == "bottom-to-top":
        i = 1
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
    key=lambda b:b[1][i], reverse=reverse))
    # return the list of sorted contours and bounding boxes
    return (cnts, boundingBoxes)

In [ ]:

LB = joblib.load('label_binarizer.pkl')
def get_letters(image,track_id):
    
    letters = []
    cnts = cv2.findContours(image.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sort_contours(cnts, method="left-to-right")[0]
    # loop over the contours
    for c in cnts:
        if cv2.contourArea(c) > 10:
            (x, y, w, h) = cv2.boundingRect(c)
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            roi = image[y:y + h, x:x + w]
            thresh = cv2.threshold(roi, 0, 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
            thresh = cv2.resize(thresh, (32, 32), interpolation = cv2.INTER_CUBIC)
            thresh = thresh.astype("float32") / 255.0
            thresh = np.expand_dims(thresh, axis=-1)
            thresh = thresh.reshape(1,32,32,1)
            ypred = cnn_model.predict(thresh)
            ypred = LB.inverse_transform(ypred)
            [x] = ypred
            letters.append(x)
    return letters, image

#plt.imshow(image)

In [ ]:
def get_word(letter):
    word = "".join(letter)
    return word

In [ ]:
def read_license_plate(image,track_id):
    letters, image = get_letters(image,track_id)
    return letters,image

In [ ]:
results = {}

# read video by index
video = cv2.VideoCapture(videos[0])

ret = True
frame_number = -1
vehicles = [2,3,5]

# read the entire video
while ret:
    ret, frame = video.read()
    frame_number += 1
    if ret:
        results[frame_number] = {}
        
        # vehicle detector
        detections = coco_model.track(frame, persist=True)[0]
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, track_id, score, class_id = detection
            if int(class_id) in vehicles and score > 0.5:
                vehicle_bounding_boxes = []
                vehicle_bounding_boxes.append([x1, y1, x2, y2, track_id, score])
                for bbox in vehicle_bounding_boxes:
                    print(bbox)
                    roi = frame[int(y1):int(y2), int(x1):int(x2)]
                    
                    # license plate detector for region of interest
                    license_plates = np_model(roi)[0]
                    # process license plate
                    for license_plate in license_plates.boxes.data.tolist():
                        plate_x1, plate_y1, plate_x2, plate_y2, plate_score, _ = license_plate
                        # crop plate from region of interest
                        plate = roi[int(plate_y1):int(plate_y2), int(plate_x1):int(plate_x2)]
                        cv2.imwrite('outputs/plates/roi/'+str(track_id) + '.jpg', plate)
                        plate_gray = cv2.cvtColor(plate, cv2.COLOR_BGR2GRAY)
                        cv2.imwrite('outputs/plates/gray/'+str(track_id)+ '.jpg', plate_gray)
                        # posterize
                        _, plate_treshold = cv2.threshold(plate_gray, 64, 255, cv2.THRESH_BINARY_INV)
                        cv2.imwrite('outputs/plates/thresh/'+str(track_id)+ '.jpg', plate_treshold)
                        # OCR
                        letters,image = read_license_plate(plate_treshold,track_id)
                        word = get_word(letters)
                        print("Word: ",word)
                        # plt.imshow(image)
                        # if plate could be read write results
                        if letters is not None:
                            results[frame_number][track_id] = {
                                'car': {
                                    'bbox': [x1, y1, x2, y2],
                                    'bbox_score': score
                                },
                                'license_plate': {
                                    'bbox': [plate_x1, plate_y1, plate_x2, plate_y2],
                                    'bbox_score': plate_score,
                                    'number': word,
                                    'text_score': plate_score
                                }
                            }

write_csv(results, './outputs/resultsCNN.csv')
video.release()


0: 384x640 2 persons, 3 cars, 113.3ms
Speed: 5.0ms preprocess, 113.3ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
[885.9718017578125, 259.9759216308594, 1918.0755615234375, 1069.1480712890625, 1.0, 0.9230321049690247]

0: 512x640 (no detections), 127.1ms
Speed: 4.1ms preprocess, 127.1ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 640)
[0.256805419921875, 368.2144470214844, 288.8199768066406, 804.2828369140625, 2.0, 0.8570149540901184]

0: 640x448 1 License_Plate, 109.8ms
Speed: 3.5ms preprocess, 109.8ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 448)
1/1 [==============================] - 0s 128ms/step
Word:  W
[155.01361083984375, 175.82943725585938, 1373.837158203125, 1058.7515869140625, 5.0, 0.7630553841590881]

0: 480x640 1 License_Plate, 131.4ms
Speed: 3.0ms preprocess, 131.4ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 [==============================] - 0s 35ms/step
Word:  TJBRGEFRE

0: 384x640 2 pe

ValueError: not enough values to unpack (expected 2, got 0)

If you liked this notebook, then do **Upvote** as it will keep me motivated in creating such kernels ahead. **Thanks!!**

### STEP 6 Visualize the Results

In [ ]:
import pandas as pd
import numpy as np

# Assuming your input data is stored in a CSV file named 'data.csv'
# You can adjust the file name or provide the data directly if it's not in a file
data = pd.read_csv('./outputs/resultsCNN.csv')

# Convert 'license_text_score' to numeric
data['license_text_score'] = pd.to_numeric(data['license_text_score'], errors='coerce')

# Calculate the total sum of license_text_score for each license_plate_number
total_license_score = data.groupby('license_plate_number')['license_text_score'].sum()

# Find the row with the maximum license_plate_score for each license_plate_number
max_license_score_row = data.loc[data.groupby('license_plate_number')['license_text_score'].idxmax()]

# Merge the two DataFrames on license_plate_number
result = pd.merge(max_license_score_row[['license_plate_number', 'track_id']], total_license_score.reset_index(),
                  on='license_plate_number', how='inner')

# Find the row with the maximum license_text_score for each track_id
max_license_score_row = result.loc[result.groupby('track_id')['license_text_score'].idxmax()]

# Display the result
print(max_license_score_row)


   license_plate_number  track_id  license_text_score
2                     B       1.0           10.942266
68                    R       2.0            2.414833
73                    W       5.0           31.788219
30                    G       6.0           30.560411
40                    H       9.0            0.404360
